In [85]:
import pandas as pd
import numpy as np

# Read the Excel files
df_550 = pd.read_excel(r"C:\Users\okkam\Desktop\labo\article 2\structural\participant_list_drive.xlsx")
df_147 = pd.read_excel(r"C:\Users\okkam\Desktop\labo\article 2\structural\Glu\participant list_glu_147.xlsx")
df_74 = pd.read_excel(r"C:\Users\okkam\Desktop\labo\article 2\structural\participant_list_longitudinal_74.xlsx")


In [92]:
# Save outputs
output_merged = r"C:\Users\okkam\Desktop\labo\article 2\structural\merged_df74_147.xlsx"
output_tp = r"C:\Users\okkam\Desktop\labo\article 2\structural\merged_with_timepoints.xlsx"

merged_df74_147.to_excel(output_merged, index=False)
merged_with_timepoints.to_excel(output_tp, index=False)

print("Saved:")
print(" -", output_merged)
print(" -", output_tp)

Saved:
 - C:\Users\okkam\Desktop\labo\article 2\structural\merged_df74_147.xlsx
 - C:\Users\okkam\Desktop\labo\article 2\structural\merged_with_timepoints.xlsx


In [91]:
# Add structural timepoints from df_550_clean
# Build pivot: index PSCID, columns Timepoint, values Full_ID_550

timepoints_550 = sorted(df_550_clean['Timepoint'].unique())
df_550_pivot = df_550_clean.pivot_table(index='PSCID', columns='Timepoint', values='Full_ID_550', aggfunc='first')

merged_with_timepoints = merged_df74_147.merge(df_550_pivot.reset_index(), on='PSCID', how='left')

print("="*80)
print("MERGE: + df_550 timepoints")
print("="*80)
print("Timepoints:", timepoints_550)
print("Shape:", merged_with_timepoints.shape)
print("\nHead:")
print(merged_with_timepoints.head(12))

MERGE: + df_550 timepoints
Timepoints: ['t00', 't02', 't04', 't06', 't08', 't10']
Shape: (115, 15)

Head:
      PSCID_CandID    PSCID  CandID  visite spectro sexe  Age_T0  Age_T2  \
0   3002498_327986  3002498  327986               4    F    72.6    74.6   
1   3025432_658178  3025432  658178               2    F    71.6    73.2   
2   3025432_658178  3025432  658178               2    F    71.6    73.2   
3   3025432_658178  3025432  658178               2    F    71.6    73.2   
4   3100205_255881  3100205  255881               4    H    73.6     NaN   
5   3123186_920577  3123186  920577               2    F    71.7    73.6   
6   3149469_790489  3149469  790489               4    H    70.2    72.2   
7   3149469_790489  3149469  790489               4    H    70.2    72.2   
8   3291977_748676  3291977  748676               6    H    87.5    89.4   
9   3388201_333084  3388201  333084               0    H    68.8    70.8   
10  3420680_878354  3420680  878354               2    H  

In [96]:
# Build baseline + next-available followup pairs for each PSCID in df_74
# Uses df_550_pivot (PSCID x Timepoint -> Full_ID_550)

# Derive ordered timepoints from pivot columns
# Convert 't02' -> 2 for sort, keep original labels
def _tp_num(tp):
    try:
        return int(tp[1:])
    except Exception:
        return 999

available_tps = [c for c in df_550_pivot.columns if isinstance(c, str) and c.startswith('t')]
timepoint_order = sorted(available_tps, key=_tp_num)

visit_to_timepoint_map = {0: 't00', 2: 't02', 4: 't04', 6: 't06', 8: 't08', 10: 't10'}

pairs_rows = []
created = 0
skipped_no_baseline = 0
skipped_no_followup = 0

for _, r in df_74.iterrows():
    pscid = int(r['PSCID'])
    visit = int(r['visite spectro']) if pd.notna(r['visite spectro']) else None
    if visit is None or visit not in visit_to_timepoint_map:
        continue
    mapped_baseline_tp = visit_to_timepoint_map[visit]

    # Ensure PSCID exists in pivot
    if pscid not in df_550_pivot.index:
        skipped_no_baseline += 1
        continue
    row_pivot = df_550_pivot.loc[pscid]

    # Relaxed baseline: choose the first available timepoint at/after mapped baseline
    try:
        start_idx = timepoint_order.index(mapped_baseline_tp)
    except ValueError:
        start_idx = 0

    baseline_id = None
    baseline_tp = None
    for tp in timepoint_order[start_idx:]:
        val = row_pivot.get(tp)
        if pd.notna(val):
            baseline_id = val
            baseline_tp = tp
            break

    if baseline_id is None:
        skipped_no_baseline += 1
        continue

    pairs_rows.append({'PSCID': pscid, 'Timepoint': baseline_tp, 'Participant_ID': baseline_id, 'Role': 'baseline'})

    # Find next available non-null timepoint after chosen baseline
    try:
        follow_start_idx = timepoint_order.index(baseline_tp)
    except ValueError:
        follow_start_idx = 0

    followup_id = None
    followup_tp = None
    for tp in timepoint_order[follow_start_idx + 1:]:
        val = row_pivot.get(tp)
        if pd.notna(val):
            followup_id = val
            followup_tp = tp
            break

    if followup_id is None:
        skipped_no_followup += 1
        continue

    pairs_rows.append({'PSCID': pscid, 'Timepoint': followup_tp, 'Participant_ID': followup_id, 'Role': 'followup'})
    created += 1

# Create the pairs dataframe (two rows per participant where followup exists)
df_pairs = pd.DataFrame(pairs_rows)

print("="*80)
print("BASELINE + FOLLOWUP PAIRS (Relaxed Baseline)")
print("="*80)
print(f"Participants with pairs created: {created}")
print(f"Skipped (no baseline at/after mapped): {skipped_no_baseline}")
print(f"Skipped (no followup after baseline): {skipped_no_followup}")
print("\nFirst 30 rows:")
print(df_pairs.head(30).to_string(index=False))

# Save to a new file to avoid permission issues
pairs_output_relaxed = r"C:\Users\okkam\Desktop\labo\article 2\structural\baseline_followup_pairs_from_74_relaxed.xlsx"
df_pairs.to_excel(pairs_output_relaxed, index=False)
print("\nSaved pairs to:", pairs_output_relaxed)

BASELINE + FOLLOWUP PAIRS (Relaxed Baseline)
Participants with pairs created: 65
Skipped (no baseline at/after mapped): 0
Skipped (no followup after baseline): 10

First 30 rows:
  PSCID Timepoint  Participant_ID     Role
3002498       t04 3002498_irm_t04 baseline
3002498       t06 3002498_irm_t06 followup
3025432       t02 3025432_irm_t02 baseline
3025432       t06 3025432_irm_t06 followup
3100205       t04 3100205_irm_t04 baseline
3100205       t08 3100205_irm_t08 followup
3123186       t02 3123186_irm_t02 baseline
3123186       t04 3123186_irm_t04 followup
3149469       t04 3149469_irm_t04 baseline
3149469       t06 3149469_irm_t06 followup
3291977       t06 3291977_irm_t06 baseline
3291977       t08 3291977_irm_t08 followup
3388201       t00 3388201_irm_t00 baseline
3388201       t04 3388201_irm_t04 followup
3420680       t02 3420680_irm_t02 baseline
3420680       t04 3420680_irm_t04 followup
3572536       t00 3572536_irm_t00 baseline
3572536       t02 3572536_irm_t02 followup
3634

In [97]:
# Unmatched participants report (reasons why a PSCID from df_74 did not yield a pair)
visit_to_timepoint_map = {0: 't00', 2: 't02', 4: 't04', 6: 't06', 8: 't08', 10: 't10'}

def _tp_num(tp):
    try:
        return int(tp[1:])
    except Exception:
        return 999

available_tps = [c for c in df_550_pivot.columns if isinstance(c, str) and c.startswith('t')]
timepoint_order = sorted(available_tps, key=_tp_num)

records = []
set_df550 = set(df_550_pivot.index)

for _, r in df_74.iterrows():
    pscid = int(r['PSCID'])
    visit = int(r['visite spectro']) if pd.notna(r['visite spectro']) else None
    if pscid not in set_df550:
        records.append({'PSCID': pscid, 'visit': visit, 'reason': 'not_in_550'})
        continue
    if visit not in visit_to_timepoint_map:
        records.append({'PSCID': pscid, 'visit': visit, 'reason': 'invalid_visit'})
        continue
    mapped_baseline_tp = visit_to_timepoint_map[visit]
    row_pivot = df_550_pivot.loc[pscid]

    # Relaxed baseline: first available at/after mapped baseline
    try:
        start_idx = timepoint_order.index(mapped_baseline_tp)
    except ValueError:
        start_idx = 0

    baseline_tp = None
    baseline_val = None
    for tp in timepoint_order[start_idx:]:
        val = row_pivot.get(tp)
        if pd.notna(val):
            baseline_tp = tp
            baseline_val = val
            break

    if baseline_val is None:
        records.append({'PSCID': pscid, 'visit': visit, 'reason': 'baseline_missing'})
        continue

    # Check follow-up availability after chosen baseline
    try:
        follow_start_idx = timepoint_order.index(baseline_tp)
    except ValueError:
        follow_start_idx = 0

    follow_found = False
    for tp in timepoint_order[follow_start_idx + 1:]:
        if pd.notna(row_pivot.get(tp)):
            follow_found = True
            break
    if not follow_found:
        records.append({'PSCID': pscid, 'visit': visit, 'reason': 'no_followup'})

unmatched_report = pd.DataFrame(records)

print("="*80)
print("UNMATCHED PARTICIPANTS REPORT (Relaxed Baseline)")
print("="*80)
if unmatched_report.empty:
    print("All df_74 participants have baseline and follow-up.")
else:
    print("Counts by reason:")
    print(unmatched_report['reason'].value_counts())
    print("\nFirst rows:")
    print(unmatched_report.head(20).to_string(index=False))

# Save report
unmatched_output = r"C:\Users\okkam\Desktop\labo\article 2\structural\unmatched_report_from_74_relaxed.xlsx"
unmatched_report.to_excel(unmatched_output, index=False)
print("\nSaved unmatched report to:", unmatched_output)

UNMATCHED PARTICIPANTS REPORT (Relaxed Baseline)
Counts by reason:
reason
no_followup    10
Name: count, dtype: int64

First rows:
  PSCID  visit      reason
5199730      2 no_followup
5237572      6 no_followup
5595871      2 no_followup
6258913      2 no_followup
6371164      2 no_followup
6412865      2 no_followup
6633412      6 no_followup
6714520      0 no_followup
8060583      6 no_followup
9234862      6 no_followup

Saved unmatched report to: C:\Users\okkam\Desktop\labo\article 2\structural\unmatched_report_from_74_relaxed.xlsx


In [90]:
# Merge df_74 (base) with df_147_clean (glutamate)
merged_df74_147 = df_74.merge(df_147_clean, on='PSCID', how='left')

has_glu = merged_df74_147['Full_ID'].notna().sum()
print("="*80)
print("MERGE: df_74 + df_147_clean")
print("="*80)
print("Shape:", merged_df74_147.shape)
print("Columns:", merged_df74_147.columns.tolist())
print("Glutamate records present:", has_glu, "/", len(merged_df74_147))
print("\nHead:")
print(merged_df74_147.head(12))

MERGE: df_74 + df_147_clean
Shape: (115, 9)
Columns: ['PSCID_CandID', 'PSCID', 'CandID', 'visite spectro', 'sexe', 'Age_T0', 'Age_T2', 'Full_ID', 'Visit_Code']
Glutamate records present: 114 / 115

Head:
      PSCID_CandID    PSCID  CandID  visite spectro sexe  Age_T0  Age_T2  \
0   3002498_327986  3002498  327986               4    F    72.6    74.6   
1   3025432_658178  3025432  658178               2    F    71.6    73.2   
2   3025432_658178  3025432  658178               2    F    71.6    73.2   
3   3025432_658178  3025432  658178               2    F    71.6    73.2   
4   3100205_255881  3100205  255881               4    H    73.6     NaN   
5   3123186_920577  3123186  920577               2    F    71.7    73.6   
6   3149469_790489  3149469  790489               4    H    70.2    72.2   
7   3149469_790489  3149469  790489               4    H    70.2    72.2   
8   3291977_748676  3291977  748676               6    H    87.5    89.4   
9   3388201_333084  3388201  333084 

In [89]:
# Clean column names for consistent merging
# df_147 columns: ['Participant ID', 'Unnamed: 1', 'Unnamed: 2']
# df_550 columns: ['Participant_ID', 'Unnamed: 1', 'Unnamed: 2']

# Standardize
df_147_clean = df_147.copy()
df_147_clean.columns = ['Full_ID', 'PSCID', 'Visit_Code']

df_550_clean = df_550.copy()
df_550_clean.columns = ['Full_ID_550', 'PSCID', 'Timepoint']

print("Renamed columns:")
print("df_147_clean:", df_147_clean.columns.tolist())
print("df_550_clean:", df_550_clean.columns.tolist())

Renamed columns:
df_147_clean: ['Full_ID', 'PSCID', 'Visit_Code']
df_550_clean: ['Full_ID_550', 'PSCID', 'Timepoint']


In [88]:
# Overview: df_74 (longitudinal subset)
print("="*80)
print("DATASET: df_74")
print("="*80)
print("Shape:", df_74.shape)
print("Columns:", df_74.columns.tolist())
print("\nDtypes:\n", df_74.dtypes)
print("\nHead:")
print(df_74.head())

DATASET: df_74
Shape: (75, 7)
Columns: ['PSCID_CandID', 'PSCID', 'CandID', 'visite spectro', 'sexe', 'Age_T0', 'Age_T2']

Dtypes:
 PSCID_CandID       object
PSCID               int64
CandID              int64
visite spectro      int64
sexe               object
Age_T0            float64
Age_T2            float64
dtype: object

Head:
     PSCID_CandID    PSCID  CandID  visite spectro sexe  Age_T0  Age_T2
0  3002498_327986  3002498  327986               4    F    72.6    74.6
1  3025432_658178  3025432  658178               2    F    71.6    73.2
2  3100205_255881  3100205  255881               4    H    73.6     NaN
3  3123186_920577  3123186  920577               2    F    71.7    73.6
4  3149469_790489  3149469  790489               4    H    70.2    72.2


In [87]:
# Overview: df_147 (glutamate)
print("="*80)
print("DATASET: df_147")
print("="*80)
print("Shape:", df_147.shape)
print("Columns:", df_147.columns.tolist())
print("\nDtypes:\n", df_147.dtypes)
print("\nHead:")
print(df_147.head())

DATASET: df_147
Shape: (147, 3)
Columns: ['Participant ID', 'Base ID', 'Visite spectro']

Dtypes:
 Participant ID    object
Base ID            int64
Visite spectro    object
dtype: object

Head:
        Participant ID  Base ID Visite spectro
0  3002498_327986_V17S  3002498           V17S
1   3025432_658178_V14  3025432            V14
2  3100205_255881_V17S  3100205           V17S
3  3123186_920577_V10S  3123186           V10S
4  3149469_790489_V17S  3149469           V17S


In [86]:
# Overview: df_550 (structural)
print("="*80)
print("DATASET: df_550")
print("="*80)
print("Shape:", df_550.shape)
print("Columns:", df_550.columns.tolist())
print("\nDtypes:\n", df_550.dtypes)
print("\nHead:")
print(df_550.head())

DATASET: df_550
Shape: (540, 3)
Columns: ['Participant_ID', 'Unnamed: 1', 'Unnamed: 2']

Dtypes:
 Participant_ID    object
Unnamed: 1         int64
Unnamed: 2        object
dtype: object

Head:
    Participant_ID  Unnamed: 1 Unnamed: 2
0  3002498_irm_t02     3002498        t02
1  3002498_irm_t04     3002498        t04
2  3002498_irm_t06     3002498        t06
3  3002498_irm_t08     3002498        t08
4  3002498_irm_t10     3002498        t10


In [98]:
# === Clean Pipeline: Read Inputs ===
import pandas as pd
import numpy as np

df_74_path = r"C:\Users\okkam\Desktop\labo\article 2\structural\participant_list_longitudinal_74.xlsx"
df_550_path = r"C:\Users\okkam\Desktop\labo\article 2\structural\participant_list_drive_550.xlsx"
df_147_path = r"C:\Users\okkam\Desktop\labo\article 2\structural\Glu\participant list_glu_147.xlsx"

df74_raw = pd.read_excel(df_74_path)
df550_raw = pd.read_excel(df_550_path)
df147_raw = pd.read_excel(df_147_path)

print("Inputs loaded:")
print(" - df74_raw:", df74_raw.shape)
print(" - df550_raw:", df550_raw.shape)
print(" - df147_raw:", df147_raw.shape)

Inputs loaded:
 - df74_raw: (75, 7)
 - df550_raw: (540, 3)
 - df147_raw: (147, 3)


In [99]:
# === Clean Pipeline: Dataset Overviews ===
print("="*80)
print("CLEAN PIPELINE OVERVIEWS")
print("="*80)
print("df74_raw columns:", df74_raw.columns.tolist())
print("df550_raw columns:", df550_raw.columns.tolist())
print("df147_raw columns:", df147_raw.columns.tolist())
print("\ndf74_raw head:\n", df74_raw.head(5))
print("\ndf550_raw head:\n", df550_raw.head(5))
print("\ndf147_raw head:\n", df147_raw.head(5))

CLEAN PIPELINE OVERVIEWS
df74_raw columns: ['PSCID_CandID', 'PSCID', 'CandID', 'visite spectro', 'sexe', 'Age_T0', 'Age_T2']
df550_raw columns: ['Participant_ID', 'Unnamed: 1', 'Unnamed: 2']
df147_raw columns: ['Participant ID', 'Base ID', 'Visite spectro']

df74_raw head:
      PSCID_CandID    PSCID  CandID  visite spectro sexe  Age_T0  Age_T2
0  3002498_327986  3002498  327986               4    F    72.6    74.6
1  3025432_658178  3025432  658178               2    F    71.6    73.2
2  3100205_255881  3100205  255881               4    H    73.6     NaN
3  3123186_920577  3123186  920577               2    F    71.7    73.6
4  3149469_790489  3149469  790489               4    H    70.2    72.2

df550_raw head:
     Participant_ID  Unnamed: 1 Unnamed: 2
0  3002498_irm_t02     3002498        t02
1  3002498_irm_t04     3002498        t04
2  3002498_irm_t06     3002498        t06
3  3002498_irm_t08     3002498        t08
4  3002498_irm_t10     3002498        t10

df147_raw head:
      

In [100]:
# === Clean Pipeline: Normalize Columns ===
df147_clean_cp = df147_raw.copy()
df147_clean_cp.columns = ['Full_ID','PSCID','Visit_Code']

df550_clean_cp = df550_raw.copy()
df550_clean_cp.columns = ['Full_ID_550','PSCID','Timepoint']

print("Normalized columns:")
print("df147_clean_cp:", df147_clean_cp.columns.tolist())
print("df550_clean_cp:", df550_clean_cp.columns.tolist())

Normalized columns:
df147_clean_cp: ['Full_ID', 'PSCID', 'Visit_Code']
df550_clean_cp: ['Full_ID_550', 'PSCID', 'Timepoint']


In [101]:
# === Clean Pipeline: Build 550 Pivot and Timepoint Order ===
def _tp_num_cp(tp):
    try:
        return int(tp[1:])
    except Exception:
        return 999

df_550_pivot_cp = df550_clean_cp.pivot_table(index='PSCID', columns='Timepoint', values='Full_ID_550', aggfunc='first')
available_tps_cp = [c for c in df_550_pivot_cp.columns if isinstance(c, str) and c.startswith('t')]
timepoint_order_cp = sorted(available_tps_cp, key=_tp_num_cp)

print("Timepoints (sorted):", timepoint_order_cp)
print("Pivot shape:", df_550_pivot_cp.shape)

Timepoints (sorted): ['t00', 't02', 't04', 't06', 't08', 't10']
Pivot shape: (285, 6)


In [102]:
# === Clean Pipeline: 74-550 Baseline+Followup Pairs (Relaxed Baseline) ===
visit_to_timepoint_map_cp = {0: 't00', 2: 't02', 4: 't04', 6: 't06', 8: 't08', 10: 't10'}

pairs_rows_cp = []
created_cp = 0
skipped_no_baseline_cp = 0
skipped_no_followup_cp = 0

for _, r in df74_raw.iterrows():
    pscid = int(r['PSCID'])
    visit = int(r['visite spectro']) if pd.notna(r['visite spectro']) else None
    if visit is None or visit not in visit_to_timepoint_map_cp:
        continue
    mapped_baseline_tp = visit_to_timepoint_map_cp[visit]

    if pscid not in df_550_pivot_cp.index:
        skipped_no_baseline_cp += 1
        continue
    row_pivot = df_550_pivot_cp.loc[pscid]

    try:
        start_idx = timepoint_order_cp.index(mapped_baseline_tp)
    except ValueError:
        start_idx = 0

    baseline_id = None
    baseline_tp = None
    for tp in timepoint_order_cp[start_idx:]:
        val = row_pivot.get(tp)
        if pd.notna(val):
            baseline_id = val
            baseline_tp = tp
            break

    if baseline_id is None:
        skipped_no_baseline_cp += 1
        continue

    pairs_rows_cp.append({'PSCID': pscid, 'Timepoint': baseline_tp, 'Participant_ID': baseline_id, 'Role': 'baseline'})

    try:
        follow_start_idx = timepoint_order_cp.index(baseline_tp)
    except ValueError:
        follow_start_idx = 0

    followup_id = None
    followup_tp = None
    for tp in timepoint_order_cp[follow_start_idx + 1:]:
        val = row_pivot.get(tp)
        if pd.notna(val):
            followup_id = val
            followup_tp = tp
            break

    if followup_id is None:
        skipped_no_followup_cp += 1
        continue

    pairs_rows_cp.append({'PSCID': pscid, 'Timepoint': followup_tp, 'Participant_ID': followup_id, 'Role': 'followup'})
    created_cp += 1

df_pairs_cp = pd.DataFrame(pairs_rows_cp)

print("="*80)
print("74-550 PAIRS (Relaxed Baseline)")
print("="*80)
print(f"Participants with pairs created: {created_cp}")
print(f"Skipped (no baseline at/after mapped): {skipped_no_baseline_cp}")
print(f"Skipped (no followup after baseline): {skipped_no_followup_cp}")
print("\nFirst 20 rows:\n", df_pairs_cp.head(20))

pairs_output_74550 = r"C:\Users\okkam\Desktop\labo\article 2\structural\baseline_followup_pairs_from_74_550.xlsx"
df_pairs_cp.to_excel(pairs_output_74550, index=False)
print("Saved pairs to:", pairs_output_74550)

74-550 PAIRS (Relaxed Baseline)
Participants with pairs created: 65
Skipped (no baseline at/after mapped): 0
Skipped (no followup after baseline): 10

First 20 rows:
       PSCID Timepoint   Participant_ID      Role
0   3002498       t04  3002498_irm_t04  baseline
1   3002498       t06  3002498_irm_t06  followup
2   3025432       t02  3025432_irm_t02  baseline
3   3025432       t06  3025432_irm_t06  followup
4   3100205       t04  3100205_irm_t04  baseline
5   3100205       t08  3100205_irm_t08  followup
6   3123186       t02  3123186_irm_t02  baseline
7   3123186       t04  3123186_irm_t04  followup
8   3149469       t04  3149469_irm_t04  baseline
9   3149469       t06  3149469_irm_t06  followup
10  3291977       t06  3291977_irm_t06  baseline
11  3291977       t08  3291977_irm_t08  followup
12  3388201       t00  3388201_irm_t00  baseline
13  3388201       t04  3388201_irm_t04  followup
14  3420680       t02  3420680_irm_t02  baseline
15  3420680       t04  3420680_irm_t04  followup


In [103]:
# === Clean Pipeline: 74-550 Unmatched Report ===
visit_to_timepoint_map_cp = {0: 't00', 2: 't02', 4: 't04', 6: 't06', 8: 't08', 10: 't10'}

records_cp = []
set_df550_cp = set(df_550_pivot_cp.index)

for _, r in df74_raw.iterrows():
    pscid = int(r['PSCID'])
    visit = int(r['visite spectro']) if pd.notna(r['visite spectro']) else None
    if pscid not in set_df550_cp:
        records_cp.append({'PSCID': pscid, 'visit': visit, 'reason': 'not_in_550'})
        continue
    if visit not in visit_to_timepoint_map_cp:
        records_cp.append({'PSCID': pscid, 'visit': visit, 'reason': 'invalid_visit'})
        continue
    mapped_baseline_tp = visit_to_timepoint_map_cp[visit]
    row_pivot = df_550_pivot_cp.loc[pscid]

    try:
        start_idx = timepoint_order_cp.index(mapped_baseline_tp)
    except ValueError:
        start_idx = 0

    baseline_tp = None
    baseline_val = None
    for tp in timepoint_order_cp[start_idx:]:
        val = row_pivot.get(tp)
        if pd.notna(val):
            baseline_tp = tp
            baseline_val = val
            break

    if baseline_val is None:
        records_cp.append({'PSCID': pscid, 'visit': visit, 'reason': 'baseline_missing'})
        continue

    follow_found = False
    try:
        follow_start_idx = timepoint_order_cp.index(baseline_tp)
    except ValueError:
        follow_start_idx = 0
    for tp in timepoint_order_cp[follow_start_idx + 1:]:
        if pd.notna(row_pivot.get(tp)):
            follow_found = True
            break
    if not follow_found:
        records_cp.append({'PSCID': pscid, 'visit': visit, 'reason': 'no_followup'})

unmatched_report_cp = pd.DataFrame(records_cp)

print("="*80)
print("74-550 UNMATCHED REPORT (Relaxed Baseline)")
print("="*80)
if unmatched_report_cp.empty:
    print("All participants have baseline and follow-up.")
else:
    print(unmatched_report_cp['reason'].value_counts())
    print("\nFirst rows:\n", unmatched_report_cp.head(20))

unmatched_output_74550 = r"C:\Users\okkam\Desktop\labo\article 2\structural\unmatched_report_from_74_550.xlsx"
unmatched_report_cp.to_excel(unmatched_output_74550, index=False)
print("Saved unmatched report to:", unmatched_output_74550)

74-550 UNMATCHED REPORT (Relaxed Baseline)
reason
no_followup    10
Name: count, dtype: int64

First rows:
      PSCID  visit       reason
0  5199730      2  no_followup
1  5237572      6  no_followup
2  5595871      2  no_followup
3  6258913      2  no_followup
4  6371164      2  no_followup
5  6412865      2  no_followup
6  6633412      6  no_followup
7  6714520      0  no_followup
8  8060583      6  no_followup
9  9234862      6  no_followup
Saved unmatched report to: C:\Users\okkam\Desktop\labo\article 2\structural\unmatched_report_from_74_550.xlsx


In [104]:
# === Clean Pipeline: 147-550 Matches (Any Timepoint Present) ===
merged_147_550_cp = df147_clean_cp.merge(df_550_pivot_cp.reset_index(), on='PSCID', how='left')

cols_timepoints = [c for c in merged_147_550_cp.columns if isinstance(c, str) and c.startswith('t')]
num_with_tp_cp = merged_147_550_cp[cols_timepoints].notna().any(axis=1).sum()
matched_147_550_cp = merged_147_550_cp[merged_147_550_cp[cols_timepoints].notna().any(axis=1)].copy()

print("="*80)
print("147-550 MATCHES")
print("="*80)
print("Merged shape:", merged_147_550_cp.shape)
print("Matched with any 550 timepoint:", num_with_tp_cp)
print("\nHead:\n", matched_147_550_cp.head(10))

matched_output_147550 = r"C:\Users\okkam\Desktop\labo\article 2\structural\matched_participants_147_with_550_timepoints.xlsx"
matched_147_550_cp.to_excel(matched_output_147550, index=False)
print("Saved 147-550 matches to:", matched_output_147550)

147-550 MATCHES
Merged shape: (147, 9)
Matched with any 550 timepoint: 142

Head:
                Full_ID    PSCID Visit_Code              t00              t02  \
0  3002498_327986_V17S  3002498       V17S              NaN  3002498_irm_t02   
1   3025432_658178_V14  3025432        V14  3025432_irm_t00  3025432_irm_t02   
2  3100205_255881_V17S  3100205       V17S              NaN              NaN   
3  3123186_920577_V10S  3123186       V10S  3123186_irm_t00  3123186_irm_t02   
4  3149469_790489_V17S  3149469       V17S  3149469_irm_t00  3149469_irm_t02   
5  3291977_748676_V24S  3291977       V24S  3291977_irm_t00              NaN   
6  3309393_500607_V31S  3309393       V31S              NaN              NaN   
7  3388201_333084_V03S  3388201       V03S  3388201_irm_t00              NaN   
8   3420680_878354_V14  3420680        V14  3420680_irm_t00  3420680_irm_t02   
9   3572536_582808_V14  3572536        V14  3572536_irm_t00  3572536_irm_t02   

               t04              t06 

In [105]:
# === Clean Pipeline: Final Summary ===
# 74-550 QC: counts by baseline/followup timepoints, unmatched reasons
# 147-550 QC: matched count and per-timepoint presence
print("="*80)
print("FINAL SUMMARY")
print("="*80)

# 74-550 pairs summary
try:
    baseline_counts = df_pairs_cp[df_pairs_cp['Role']=='baseline']['Timepoint'].value_counts().sort_index()
    followup_counts = df_pairs_cp[df_pairs_cp['Role']=='followup']['Timepoint'].value_counts().sort_index()
    print("74-550: paired participants:", df_pairs_cp[df_pairs_cp['Role']=='baseline'].shape[0])
    print("74-550: baseline counts by timepoint:\n", baseline_counts)
    print("74-550: follow-up counts by timepoint:\n", followup_counts)
except Exception as e:
    print("74-550 pairs summary error:", e)

# 74-550 unmatched summary
try:
    print("74-550: unmatched counts by reason:\n", unmatched_report_cp['reason'].value_counts())
except Exception as e:
    print("74-550 unmatched summary error:", e)

# 147-550 matches summary
try:
    cols_timepoints = [c for c in matched_147_550_cp.columns if isinstance(c, str) and c.startswith('t')]
    per_tp_counts = {tp: matched_147_550_cp[tp].notna().sum() for tp in sorted(cols_timepoints)}
    dist_tp_present = matched_147_550_cp[cols_timepoints].notna().sum(axis=1).value_counts().sort_index()
    print("147-550: matched rows:", len(matched_147_550_cp))
    print("147-550: per-timepoint presence counts:")
    for tp, cnt in per_tp_counts.items():
        print(f"  {tp}: {cnt}")
    print("147-550: distribution of number of timepoints present per row:\n", dist_tp_present)
except Exception as e:
    print("147-550 matches summary error:", e)

# Saved file paths
try:
    print("\nSaved files:")
    print(" - 74-550 pairs:", pairs_output_74550)
    print(" - 74-550 unmatched:", unmatched_output_74550)
    print(" - 147-550 matches:", matched_output_147550)
except Exception as e:
    print("Saved paths reporting error:", e)

FINAL SUMMARY
74-550: paired participants: 75
74-550: baseline counts by timepoint:
 Timepoint
t00    26
t02    18
t04    15
t06    15
t08     1
Name: count, dtype: int64
74-550: follow-up counts by timepoint:
 Timepoint
t02    24
t04    12
t06    16
t08    12
t10     1
Name: count, dtype: int64
74-550: unmatched counts by reason:
 reason
no_followup    10
Name: count, dtype: int64
147-550: matched rows: 142
147-550: per-timepoint presence counts:
  t00: 120
  t02: 104
  t04: 68
  t06: 71
  t08: 46
  t10: 3
147-550: distribution of number of timepoints present per row:
 1    24
2    44
3    19
4    32
5    23
Name: count, dtype: int64

Saved files:
 - 74-550 pairs: C:\Users\okkam\Desktop\labo\article 2\structural\baseline_followup_pairs_from_74_550.xlsx
 - 74-550 unmatched: C:\Users\okkam\Desktop\labo\article 2\structural\unmatched_report_from_74_550.xlsx
 - 147-550 matches: C:\Users\okkam\Desktop\labo\article 2\structural\matched_participants_147_with_550_timepoints.xlsx
